#Mercari Price Suggestion Challenge

https://www.kaggle.com/c/mercari-price-suggestion-challenge


In [1]:
import numpy as np
import pandas as pd
import math
from time import time
import datetime
from sklearn.model_selection import GridSearchCV

def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5
#Source: https://www.kaggle.com/marknagelberg/rmsle-function


In [2]:
from sklearn.impute import SimpleImputer 
def get_data(dir):
    t=time()
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    train=pd.read_csv(dir+"train.tsv",sep='\t').fillna(" ")
    test=pd.read_csv(dir+"test.tsv",sep='\t').fillna(" ")
    print(train.shape)
    #print(train.head(3))
#     print(train.describe())
    print(test.shape)
    #print(test.head(3))
#     print(test.describe())


#     train.loc[train['price'] <=0] = 1
#     test.loc[test['price'] <=0] = 1
#     print(train.describe())
#     print(test.describe())
    print(" {0} 秒完数据读入".format(time() - t))
 
    t=time()
    #此时归一化，tfidf会自私归一化，而且耗时很长
    x_train=train[["item_condition_id","shipping"]]
    x_test=test[["item_condition_id","shipping"]]
    y_train=train["price"]
    y_test=test["price"]
    
    x_train= pd.DataFrame(SimpleImputer ().fit_transform(x_train))
#     y_train= Imputer().fit_transform(y_train)
    x_test= pd.DataFrame(SimpleImputer ().fit_transform(x_test))
#     y_test= Imputer().fit_transform(y_test)    
    t = time()
    print(x_train.shape)
    #print(x_train.head(3))
    
#     x_train,x_test=add_tfidf(x_train,x_test,train,test)
    x_train,x_test=add_word2vec(x_train,x_test,train,test)
    x_train= pd.DataFrame(SimpleImputer ().fit_transform(x_train))
#     y_train= Imputer().fit_transform(y_train)
    x_test= pd.DataFrame(SimpleImputer ().fit_transform(x_test))
#     y_test= Imputer().fit_transform(y_test)

    x_train = pd.DataFrame(MinMaxScaler().fit_transform(x_train))
    x_test = pd.DataFrame(MinMaxScaler().fit_transform(x_test))

    # print(x_train.head(3))
    print(x_train.shape)
    # print(x_test.head(3))
    print(x_test.shape)
    print(" {0} 秒完成特征提取".format(time() - t))
    t = time()

    return x_train,y_train,x_test,y_test

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
def add_tfidf(x_train,x_test,train,test):
# 不可四列一起训练，name band 太稀少了  不划算
    text_columns=["category_name","brand_name","name","item_description"]
    max_features=[1000,10000,20000,100000]
    max_features=[100,1000,2000,10000]
    tfidfer=[None]*4
    for i in range(4):
        tfidfer[i]=TfidfVectorizer(max_features=max_features[i],ngram_range=(1,1)).fit(train[text_columns[i]])
        a=tfidfer[i].transform(train[text_columns[i]])
        b=pd.DataFrame(a.toarray(),columns=tfidfer[i].get_feature_names())
        # print(tfidfer[i].get_feature_names())
        print(b.shape)
     #   print(b.head(3))
        x_train=x_train.join(b,rsuffix="_%s_" %(text_columns[i]))

        c=tfidfer[i].transform(test[text_columns[i]])
        d=pd.DataFrame(c.toarray(),columns=tfidfer[i].get_feature_names())
        x_test=x_test.join(d,rsuffix="_%s_"%(text_columns[i]))

    return x_train,x_test

In [4]:
from gensim.models import Word2Vec
def add_word2vec(x_train,x_test,train,test):
# 不可四列一起训练，name band 太稀少了  不划算
    text_columns=["category_name","brand_name","name","item_description"]
    num_features=[50,50,100,100]
    for i in range(4):
        series=train[text_columns[i]].astype(str)
        print(type(series))
#         print(series.head(2))
        lines=series.values.tolist()
        print(type(lines[0]))
#         print(lines[0:3])
#             cutWords_list = [k.split() for k in file.readlines()]
        sentences=[line.split() for line in lines]
    #    sentences=train["item_description"]
    #    sentences=x_train[["name","item_description"]
        print(type(sentences))
        # 模型参数
        num_feature = num_features[i]    # Word vector dimensionality                      
        min_word_count = 40   # Minimum word count                        
        num_workers = 4       # Number of threads to run in parallel
        context = 10          # Context window size                                                                                    
        downsampling = 1e-3   # Downsample setting for frequent words
        model = Word2Vec(sentences, workers=num_workers, \
                    size=num_feature, min_count=min_word_count, \
                    window=context, sample=downsampling)
#         model=Word2Vec(sentences,size=num_feature)
        b=pd.DataFrame(getAvgFeatureVecs(sentences,model,num_feature))
        
        x_train=x_train.join(b,rsuffix="_%s_" %(text_columns[i]))
        
        series=test[text_columns[i]].astype(str)
        lines=series.values.tolist()
        sentences=[line.split() for line in lines]

        
        d=pd.DataFrame(getAvgFeatureVecs(sentences,model,num_feature))
        x_test=x_test.join(d,rsuffix="_%s_"%(text_columns[i]))

        print(x_train.shape)
#         print(x_train.head(3))
        print(x_test.shape)
#         print(x_test.head(3))
    return x_train,x_test

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
def makeFeatureVec(sentence, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float64")
    nwords = 0.
    index2word_set = set(model.wv.index2word)
    for word in sentence:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])

    featureVec = np.divide(featureVec, nwords)
    return featureVec

def getAvgFeatureVecs(sentences, model, num_feature):
    counter = 0
    vecs = np.zeros((len(sentences), num_feature), dtype="float64")

    for sentence in sentences:
#         if counter % 10000 == 0:
#             print("sentence %d of %d" % (counter, len(sentences)))

        vecs[counter] = makeFeatureVec(sentence, model, num_feature)
        counter += 1

    return vecs

In [ ]:

def model_predict(gen_models,x_train,y_train,x_test,y_test):

    models=gen_models()
    result=[["model","rmsle"],["time"]]
    for model in models:
        t=time()
        print()
        print(datetime.datetime.now())        
        model=model.fit(x_train,y_train)
        y_predict=model.predict(x_test)
        y_predict[y_predict<=0] = 1
        error=rmsle(y_test,y_predict)
        print(datetime.datetime.now())
        print(model.__module__+"训练结束")
        print("rmsle:%f" %error)
#         print("训练结束"+model.get_params)
        eclapsed=time() - t
        print(" {0} 秒耗时总计".format(eclapsed))
#         result.append([model.__name__,error,eclapsed])
        result.append(["name",error,eclapsed])
        
#         import matplotlib as mpl
#         import matplotlib.pyplot as plt
#         axis=np.arange(len(y_test))
#         mpl.rcParams['font.sans-serif'] = ['simHei']
#         mpl.rcParams['axes.unicode_minus'] = False
#         plt.figure(facecolor='w')
#         plt.plot(t, y_test, 'r-', lw=2, label='真实值')
#         plt.plot(t, y_predict, 'g-', lw=2, label='估计值')
#         plt.legend(loc='best')
# #         plt.title(model.___name_, fontsize=18)
#         plt.xlabel('商品编号', fontsize=15)
#         plt.ylabel('商品价格', fontsize=15)
#         plt.grid()
#         plt.show()



模型








In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso, RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
def gen_linear():
    linear=LinearRegression()
    ridge=Ridge()
    lasso=Lasso()
#     linear_pipeline=Pipeline([
#             ('poly', PolynomialFeatures()),
#             ('linear', ElasticNetCV(alphas=np.logspace(-3, 2, 10), l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
#                                     fit_intercept=False))])
#     return [linear,ridge,lasso,linear_pipeline]
    return [linear,ridge,lasso]


In [ ]:
from sklearn.tree import DecisionTreeRegressor
def gen_tree():
    dtr=DecisionTreeRegressor(criterion='mse', max_depth=5)
    return [dtr]

In [ ]:
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
import xgboost as xgb
def gen_ensem():
    ridge = RidgeCV(alphas=np.logspace(-3, 2, 3), fit_intercept=False)
    ridged = Pipeline([('poly', PolynomialFeatures(degree=degree)), ('Ridge', ridge)])
    bagr= BaggingRegressor(ridged, n_estimators=n_estimators, max_samples=max_samples)
    rfr = RandomForestRegressor(n_estimators=10, criterion='entropy', max_depth=5, oob_score=True)
    dbr= GradientBoostingRegressor(n_estimators=10, learning_rate=0.1, max_depth=2)
    adar = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=10, learning_rate=0.1)

    params={"max_depth":3,"learning_rate":0.1,"n_estimators":10,"silent":0,"objective":"reg:linear"}
    xgbr=xgb.XGBRegressor(**params)
    parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}
    gsxgbr = GridSearchCV(model,parameters,cv = 5,n_jobs = 5,verbose=True)
    return [bagr,rfr,dbr,adar,xgbr,gsxgbr]

In [ ]:
from sklearn import svm
def gen_svm():
    svr_linear = svm.SVR(kernel='linear', C=100)
    svr_poly = svm.SVR(kernel='poly', degree=3, C=100)
    svr_rbf = svm.SVR(kernel='rbf', gamma=0.2, C=100)
    return [svr_linear,svr_rbf,svr_poly]

In [ ]:
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
def gen_em():
    gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=0)
    dpgmm = BayesianGaussianMixture(n_components=n_components, covariance_type='full', max_iter=1000, n_init=5,
                                    weight_concentration_prior_type='dirichlet_process', weight_concentration_prior=0.1)
    return [gmm,dpgmm]

In [ ]:
if __name__=="__main__":
    t0 = time()
    t = t0
    print("开始时间: ")
    print(datetime.datetime.now())
    dir = r"C:/Users/zen/code/Mercari Price Suggestion Challenge/data/"
    dir = r"./data/"
    x_train,y_train,x_test,y_test=get_data(dir)
    # x_train,y_train,x_test,y_test=get_word2vec(dir)
    # x_train,y_train,x_test,y_test=get_brtt(dir)
    #y_predict=xgb_predict(x_train,y_train,x_test)
    #models: xgb linear rindge lasso decissiontree randomforest svr gmm 
    #models: linear tree svm em
    gens=[gen_linear,gen_tree,gen_svm,gen_em]
#     gens=[gen_linear]
    for gen in gens:
        model_predict(gen,x_train,y_train,x_test,y_test)
    # y_predict=xgb_model(x_train,y_train,x_test)
    # print("rmsle:%f" %rmsle(y_test,y_predict))
    # print(" {0} 秒耗时总计".format(time() - t))

开始时间: 
2019-03-02 22:18:41.463831
(36689, 8)
(9173, 8)
 0.19148778915405273 秒完数据读入
(36689, 2)
<class 'pandas.core.series.Series'>
<class 'str'>
<class 'list'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(36689, 52)
(9173, 52)
<class 'pandas.core.series.Series'>
<class 'str'>
<class 'list'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


(36689, 102)
(9173, 102)
<class 'pandas.core.series.Series'>
<class 'str'>
<class 'list'>
(36689, 202)
(9173, 202)
<class 'pandas.core.series.Series'>
<class 'str'>
<class 'list'>
(36689, 302)
(9173, 302)
(36689, 302)
(9173, 302)
 18.9547860622406 秒完成特征提取

2019-03-02 22:19:00.623072
2019-03-02 22:19:01.183588
sklearn.linear_model.base训练结束
rmsle:2.700371
 0.561514139175415 秒耗时总计

2019-03-02 22:19:01.184586
2019-03-02 22:19:01.484467
sklearn.linear_model.ridge训练结束
rmsle:1.573226
 0.2998814582824707 秒耗时总计

2019-03-02 22:19:01.484467
2019-03-02 22:19:02.032020
sklearn.linear_model.coordinate_descent训练结束
rmsle:0.943650
 0.5475528240203857 秒耗时总计

2019-03-02 22:19:02.032020
2019-03-02 22:19:05.783668
sklearn.tree.tree训练结束
rmsle:0.807894
 3.75164794921875 秒耗时总计

2019-03-02 22:19:05.783668
